In [ ]:
!pip install openpyxl

In [ ]:
!pip install xlrd

In [ ]:
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip

In [ ]:
!unzip v0.9.2.zip

In [ ]:
cd fastText-0.9.2

In [ ]:
!make

c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/args.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/autotune.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/matrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/dictionary.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/loss.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/productquantizer.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/densematrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/quantmatrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/vector.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/model.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/utils.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -

In [ ]:
!./fasttext

usage: fasttext <command> <args>

The commands supported by fasttext are:

  supervised              train a supervised classifier
  quantize                quantize a model to reduce the memory usage
  test                    evaluate a supervised classifier
  test-label              print labels with precision and recall scores
  predict                 predict most likely labels
  predict-prob            predict most likely labels with probabilities
  skipgram                train a skipgram model
  cbow                    train a cbow model
  print-word-vectors      print word vectors given a trained model
  print-sentence-vectors  print sentence vectors given a trained model
  print-ngrams            print ngrams given a trained model and word
  nn                      query for nearest neighbors
  analogies               query for analogies
  dump                    dump arguments,dictionary,input/output vectors



In [ ]:
import pandas as pd
import numpy as np
from io import StringIO
import csv
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from io import StringIO
import csv
from google.colab import drive
drive.mount('/content/drive')

# messages = pd.read_csv("/content/drive/My Drive/Colab Datasets/super_drift_fast.csv", encoding='latin-1')
messages = pd.read_csv("/content/drive/My Drive/Colab Datasets/super23_lbl.csv", encoding='latin-1')
# messages = pd.read_excel("/content/drive/My Drive/Colab Datasets/punny1.xlsx")
# messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
# messages = pd.read_excel('test_punny.xlsx')
messages.columns = ["label", "message"]
print (messages.head())

messages['label']=['__label__'+ s for s in messages['label']]
messages['message']= messages['message'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)
messages.to_csv(r'super23.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
print (messages.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  label                                            message
0  spam  YOU HAVE WON! As a valued Vodafone customer ou...
1  spam  Guinness Record! World?s most pierced women! C...
2  spam  U have a secret admirer. REVEAL who thinks U R...
3  spam  U have a secret admirer who is looking 2 make ...
4  spam  FREE for 1st week! No1 Nokia tone 4 ur mob eve...
           label                                            message
0  __label__spam  YOU HAVE WON! As a valued Vodafone customer ou...
1  __label__spam  Guinness Record! World?s most pierced women! C...
2  __label__spam  U have a secret admirer. REVEAL who thinks U R...
3  __label__spam  U have a secret admirer who is looking 2 make ...
4  __label__spam  FREE for 1st week! No1 Nokia tone 4 ur mob eve...


In [ ]:
cat "super23.txt" | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > "super23_preprocessed.txt"

In [ ]:
!head -n 52588 "super23_preprocessed.txt" > "super23_train_evl.txt"

In [ ]:
!tail -n 13148 "super23_preprocessed.txt" > "super23_test_evl.txt"

In [ ]:
%%time
!./fasttext supervised -input "super23_train_evl.txt" -output scam_detection_model5a -lr 1.0 -epoch 25
# !./fasttext supervised -input "super23_train.txt" -output scam_detection_model5a -lr 1.0 -epoch 25 -wordNgrams 3

Read 1M words
Number of words:  54120
Number of labels: 2
Progress: 100.0% words/sec/thread:  397876 lr:  0.000000 avg.loss:  0.005370 ETA:   0h 0m 0s
CPU times: user 141 ms, sys: 9.38 ms, total: 151 ms
Wall time: 11.2 s


In [ ]:
!./fasttext test scam_detection_model5a.bin "super23_test_evl.txt"

N	13148
P@1	0.927
R@1	0.927


In [ ]:
!./fasttext predict scam_detection_model5a.bin "super23_test_evl.txt" > "/content/drive/My Drive/Colab Datasets/sp23e.txt"

# New Section

In [ ]:
import pandas as pd
messages = pd.read_csv("/content/drive/My Drive/Colab Datasets/output.csv", encoding='latin-1')
messages.columns = ["act", "pred"]
print (messages.tail(3))

       act pred
13145  ham  ham
13146  ham  ham
13147  ham  ham


In [ ]:
from sklearn import metrics
# print(metrics.classification_report(messages["act"], messages["pred"], digits=4))
print(metrics.confusion_matrix(messages["act"], messages["pred"]))
# print("Precision: ", metrics.precision_score(messages["act"], messages["pred"]))
# print("Recall: ", metrics.recall_score(messages["act"], messages["pred"]))
# Printing the Overall Accuracy of the model
print("Accuracy of the model : {0:0.3f}".format(metrics.accuracy_score(messages["act"], messages["pred"])))

[[ 1884    34]
 [  930 10300]]
Accuracy of the model : 0.927
